In [23]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

def scrape_sanremo_edizioni_table(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', class_='wikitable')
        if table:
            try:
                df = pd.read_html(StringIO(str(table)))[0]
                return df
            except Exception as e:
                print(f"Errore nella lettura della tabella con pandas: {e}")
                return None
        else:
            print(f"Tabella 'wikitable' non trovata in {url}")
            return None
    else:
        print(f"Errore nel recupero di {url}: {response.status_code}")
        return None

url = 'https://it.wikipedia.org/wiki/Festival_di_Sanremo#Edizioni'
sanremo_edizioni_df = scrape_sanremo_edizioni_table(url)

if sanremo_edizioni_df is not None:
    display(sanremo_edizioni_df)

Edizione  Anno           Periodo Serate Partecipanti          \
   Edizione  Anno           Periodo Serate     Campioni           
   Edizione  Anno           Periodo Serate      Artisti Canzoni   
0        1ª  1951   29 - 31 gennaio      3            3      20   
1        2ª  1952   28 - 30 gennaio      3            5      20   
2        3ª  1953   29 - 31 gennaio      3           10      20   
3        4ª  1954   28 - 30 gennaio      3           12      20   
4        5ª  1955   27 - 29 gennaio      3           15      16   
..      ...   ...               ...    ...          ...     ...   
70      71ª  2021       2 - 6 marzo      5           26      26   
71      72ª  2022   1º - 5 febbraio      5           25      25   
72      73ª  2023   7 - 11 febbraio      5           28      28   
73      74ª  2024   6 - 10 febbraio      5           30      30   
74      75ª  2025  11 - 15 febbraio      5           29      29   

                        Conduzione                                    \
   Nuove proposte       Conduzione                                     
   Nuove proposte       Conduzione     Conduzione.1     Conduzione.2   
0    Non presenti  Nunzio Filogamo  Nunzio Filogamo  Nunzio Filogamo   
1    Non presenti  Nunzio Filogamo  Nunzio Filogamo  Nunzio Filogamo   
2    Non presenti  Nunzio Filogamo  Nunzio Filogamo  Nunzio Filogamo   
3    Non presenti  Nunzio Filogamo  Nunzio Filogamo  Nunzio Filogamo   
4    Non presenti    Armando Pizzo    Armando Pizzo    Armando Pizzo   
..            ...              ...              ...              ...   
70              8          Amadeus          Amadeus     Fiorello[92]   
71   Non presenti          Amadeus          Amadeus          Amadeus   
72   Non presenti          Amadeus          Amadeus   Gianni Morandi   
73   Non presenti          Amadeus          Amadeus          Amadeus   
74              4      Carlo Conti      Carlo Conti      Carlo Conti   

                    Direzione artistica                       Sede  
                    Direzione artistica                       Sede  
       Conduzione.3 Direzione artistica                       Sede  
0   Nunzio Filogamo        Giulio Razzi          Casinò di Sanremo  
1   Nunzio Filogamo        Giulio Razzi          Casinò di Sanremo  
2   Nunzio Filogamo        Giulio Razzi          Casinò di Sanremo  
3   Nunzio Filogamo        Giulio Razzi          Casinò di Sanremo  
4     Armando Pizzo        Giulio Razzi          Casinò di Sanremo  
..              ...                 ...                        ...  
70     Fiorello[92]             Amadeus  Teatro Ariston di Sanremo  
71          Amadeus             Amadeus  Teatro Ariston di Sanremo  
72   Gianni Morandi             Amadeus  Teatro Ariston di Sanremo  
73          Amadeus             Amadeus  Teatro Ariston di Sanremo  
74      Carlo Conti         Carlo Conti  Teatro Ariston di Sanremo  

[75 rows x 13 columns]

In [24]:
# 1. Tieni solo il primo livello di intestazione
sanremo_edizioni_df.columns = sanremo_edizioni_df.columns.get_level_values(0)

# 2. Rimuovi le prime due righe se sono intestazioni ripetute
if sanremo_edizioni_df.iloc[0].str.contains("Edizione|Anno", na=False).any():
    sanremo_edizioni_df = sanremo_edizioni_df.iloc[2:].reset_index(drop=True)

# 3. Continua con la pulizia come nel codice precedente:
# Rinomina colonne, unifica conduzione, rimuovi note [xx], ecc.

# Esempio rapido per rinominare e unire le colonne:
sanremo_edizioni_df.columns = [
    'Edizione', 'Anno', 'Periodo', 'Serate', 'Campioni', 'Artisti', 'Canzoni',
    'Presentatore', 'Conduttore1', 'Conduttore2', 'Conduttore3', 'Direzione_artistica', 'Sede'
]

# Unifica la conduzione
sanremo_edizioni_df['Conduzione'] = sanremo_edizioni_df[[
    'Conduttore1', 'Conduttore2', 'Conduttore3'
]].apply(lambda row: ', '.join(row.dropna().astype(str).unique()), axis=1)

# Rimuovi le colonne conduttori originali
sanremo_edizioni_df.drop(columns=['Conduttore1', 'Conduttore2', 'Conduttore3'], inplace=True)

# Pulisci note tipo [92]
sanremo_edizioni_df.replace(r'\[\d+\]', '', regex=True, inplace=True)

# Converti Anno a numerico
sanremo_edizioni_df['Anno'] = pd.to_numeric(sanremo_edizioni_df['Anno'], errors='coerce')

# Reset index e ordina
sanremo_edizioni_df.dropna(how='all', inplace=True)
sanremo_edizioni_df.reset_index(drop=True, inplace=True)
sanremo_edizioni_df.sort_values('Anno', inplace=True)

In [25]:
display(sanremo_edizioni_df)

,Edizione,Anno,Periodo,Serate,Campioni,Artisti,Canzoni,Presentatore,Direzione_artistica,Sede,Conduzione
0,1ª,1951,29 - 31 gennaio,3,3,20,Non presenti,Nunzio Filogamo,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo
1,2ª,1952,28 - 30 gennaio,3,5,20,Non presenti,Nunzio Filogamo,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo
2,3ª,1953,29 - 31 gennaio,3,10,20,Non presenti,Nunzio Filogamo,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo
3,4ª,1954,28 - 30 gennaio,3,12,20,Non presenti,Nunzio Filogamo,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo
4,5ª,1955,27 - 29 gennaio,3,15,16,Non presenti,Armando Pizzo,Giulio Razzi,Casinò di Sanremo,Armando Pizzo
...,...,...,...,...,...,...,...,...,...,...,...
70,71ª,2021,2 - 6 marzo,5,26,26,8,Amadeus,Amadeus,Teatro Ariston di Sanremo,"Amadeus, Fiorello"
71,72ª,2022,1º - 5 febbraio,5,25,25,Non presenti,Amadeus,Amadeus,Teatro Ariston di Sanremo,Amadeus
72,73ª,2023,7 - 11 febbraio,5,28,28,Non presenti,Amadeus,Amadeus,Teatro Ariston di Sanremo,"Amadeus, Gianni Morandi"
73,74ª,2024,6 - 10 febbraio,5,30,30,Non presenti,Amadeus,Amadeus,Teatro Ariston di Sanremo,Amadeus


In [26]:
sanremo_edizioni_df['Anno'] = sanremo_edizioni_df['Anno'].astype(int)

In [27]:
sanremo_edizioni_df.loc[sanremo_edizioni_df['Anno'] == 1968, 'Periodo'] = '31 gennaio - 2 febbraio'

In [28]:
# Sostituisce tutti i tipi di trattini con quello standard '-'
sanremo_edizioni_df['Periodo'] = sanremo_edizioni_df['Periodo'].str.replace(r'[–—‒−]', '-', regex=True)

In [29]:
# Divide la colonna "Periodo" in due colonne
sanremo_edizioni_df[['giorno_inizio', 'giorno_fine']] = sanremo_edizioni_df['Periodo'].str.split('-', expand=True)

# Pulisce caratteri speciali e spazi
sanremo_edizioni_df['giorno_inizio'] = sanremo_edizioni_df['giorno_inizio'].str.replace('º', '', regex=False).str.strip()
sanremo_edizioni_df['giorno_fine'] = sanremo_edizioni_df['giorno_fine'].str.replace('º', '', regex=False).str.strip()

# Estrai giorno e mese per inizio
sanremo_edizioni_df[['giorno_inizio_num', 'mese_inizio']] = sanremo_edizioni_df['giorno_inizio'].str.split(' ', expand=True)

# Estrai giorno e (eventualmente) mese per fine
giorno_fine_split = sanremo_edizioni_df['giorno_fine'].str.split(' ', expand=True)
sanremo_edizioni_df['giorno_fine_num'] = giorno_fine_split[0]
sanremo_edizioni_df['mese_fine'] = giorno_fine_split[1]

# Se mese_fine è NaN, copia il mese da inizio
sanremo_edizioni_df['mese_fine'] = sanremo_edizioni_df['mese_fine'].fillna(sanremo_edizioni_df['mese_inizio'])
sanremo_edizioni_df['mese_inizio'] = sanremo_edizioni_df['mese_inizio'].fillna(sanremo_edizioni_df['mese_fine'])

# Crea le nuove colonne con date in formato stringa
sanremo_edizioni_df['data_inizio'] = sanremo_edizioni_df['giorno_inizio_num'] + ' ' + sanremo_edizioni_df['mese_inizio'] + ' ' + sanremo_edizioni_df['Anno'].astype(str)
sanremo_edizioni_df['data_fine'] = sanremo_edizioni_df['giorno_fine_num'] + ' ' + sanremo_edizioni_df['mese_fine']+ ' ' + sanremo_edizioni_df['Anno'].astype(str)

In [30]:
display(sanremo_edizioni_df)

,Edizione,Anno,Periodo,Serate,Campioni,Artisti,Canzoni,Presentatore,Direzione_artistica,Sede,Conduzione,giorno_inizio,giorno_fine,giorno_inizio_num,mese_inizio,giorno_fine_num,mese_fine,data_inizio,data_fine
0,1ª,1951,29 - 31 gennaio,3,3,20,Non presenti,Nunzio Filogamo,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo,29,31 gennaio,29,gennaio,31,gennaio,29 gennaio 1951,31 gennaio 1951
1,2ª,1952,28 - 30 gennaio,3,5,20,Non presenti,Nunzio Filogamo,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo,28,30 gennaio,28,gennaio,30,gennaio,28 gennaio 1952,30 gennaio 1952
2,3ª,1953,29 - 31 gennaio,3,10,20,Non presenti,Nunzio Filogamo,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo,29,31 gennaio,29,gennaio,31,gennaio,29 gennaio 1953,31 gennaio 1953
3,4ª,1954,28 - 30 gennaio,3,12,20,Non presenti,Nunzio Filogamo,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo,28,30 gennaio,28,gennaio,30,gennaio,28 gennaio 1954,30 gennaio 1954
4,5ª,1955,27 - 29 gennaio,3,15,16,Non presenti,Armando Pizzo,Giulio Razzi,Casinò di Sanremo,Armando Pizzo,27,29 gennaio,27,gennaio,29,gennaio,27 gennaio 1955,29 gennaio 1955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71ª,2021,2 - 6 marzo,5,26,26,8,Amadeus,Amadeus,Teatro Ariston di Sanremo,"Amadeus, Fiorello",2,6 marzo,2,marzo,6,marzo,2 marzo 2021,6 marzo 2021
71,72ª,2022,1º - 5 febbraio,5,25,25,Non presenti,Amadeus,Amadeus,Teatro Ariston di Sanremo,Amadeus,1,5 febbraio,1,febbraio,5,febbraio,1 febbraio 2022,5 febbraio 2022
72,73ª,2023,7 - 11 febbraio,5,28,28,Non presenti,Amadeus,Amadeus,Teatro Ariston di Sanremo,"Amadeus, Gianni Morandi",7,11 febbraio,7,febbraio,11,febbraio,7 febbraio 2023,11 febbraio 2023
73,74ª,2024,6 - 10 febbraio,5,30,30,Non presenti,Amadeus,Amadeus,Teatro Ariston di Sanremo,Amadeus,6,10 febbraio,6,febbraio,10,febbraio,6 febbraio 2024,10 febbraio 2024


In [31]:
missing_values = sanremo_edizioni_df['giorno_fine'].isnull().sum()
print("Numero di valori mancanti nella colonna giorno_fine:", missing_values)
missing_values = sanremo_edizioni_df['giorno_inizio'].isnull().sum()
print("Numero di valori mancanti nella colonna giorno_inizio:", missing_values)

Numero di valori mancanti nella colonna giorno_fine: 0
Numero di valori mancanti nella colonna giorno_inizio: 0


In [32]:
sanremo_edizioni_df = sanremo_edizioni_df.drop(columns=['giorno_inizio', 'giorno_fine', 'giorno_inizio_num', 'mese_inizio', 'giorno_fine_num', 'mese_fine', 'Canzoni', 'Presentatore'])

In [35]:
display(sanremo_edizioni_df)

,Edizione,Anno,Periodo,Serate,Campioni,Artisti,Direzione_artistica,Sede,Conduzione,data_inizio,data_fine
0,1,1951,29 - 31 gennaio,3,3,20,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo,29 gennaio 1951,31 gennaio 1951
1,2,1952,28 - 30 gennaio,3,5,20,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo,28 gennaio 1952,30 gennaio 1952
2,3,1953,29 - 31 gennaio,3,10,20,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo,29 gennaio 1953,31 gennaio 1953
3,4,1954,28 - 30 gennaio,3,12,20,Giulio Razzi,Casinò di Sanremo,Nunzio Filogamo,28 gennaio 1954,30 gennaio 1954
4,5,1955,27 - 29 gennaio,3,15,16,Giulio Razzi,Casinò di Sanremo,Armando Pizzo,27 gennaio 1955,29 gennaio 1955
...,...,...,...,...,...,...,...,...,...,...,...
70,71,2021,2 - 6 marzo,5,26,26,Amadeus,Teatro Ariston di Sanremo,"Amadeus, Fiorello",2 marzo 2021,6 marzo 2021
71,72,2022,1º - 5 febbraio,5,25,25,Amadeus,Teatro Ariston di Sanremo,Amadeus,1 febbraio 2022,5 febbraio 2022
72,73,2023,7 - 11 febbraio,5,28,28,Amadeus,Teatro Ariston di Sanremo,"Amadeus, Gianni Morandi",7 febbraio 2023,11 febbraio 2023
73,74,2024,6 - 10 febbraio,5,30,30,Amadeus,Teatro Ariston di Sanremo,Amadeus,6 febbraio 2024,10 febbraio 2024


In [34]:
sanremo_edizioni_df['Edizione'] = sanremo_edizioni_df['Edizione'].astype(str).str.replace('ª', '', regex=False).str.replace('º', '', regex=False)

In [39]:
sanremo_edizioni_df.to_excel('dati-edizioni-sanremo-1951-2023.xlsx',index=False)